In [148]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [149]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [150]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [151]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [152]:
data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
print(data_pk2_path)
print(media_pk2_path)

c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Data.pk2
c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Media.pk2


In [153]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [154]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [155]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [156]:
_media_file_stream = open(media_pk2_path, 'rb')

In [157]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [158]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [159]:
blowfish_checksum_decoded = "Joymax Pak File"

In [160]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [161]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [162]:
Root = read_blocks_at(256)

In [163]:
blocks_in_memory = {
  "": Root
}

In [164]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [165]:
server_dep = os.path.join('server_dep', 'silkroad', 'textdata')
server_dep

'server_dep\\silkroad\\textdata'

In [166]:
quest_reward_items_data = []
quest_rewards_data = dict()
quest_data = dict()

## refquestrewarditems.txt

In [167]:
quest_reward_items = os.path.join(server_dep, 'refquestrewarditems.txt')

In [168]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  


In [169]:
csv_file_path = 'item_data.csv'

df = pd.read_csv(csv_file_path, encoding='utf-8', index_col=0)

In [170]:
df

,Service,ID,CodeName,ObjName,NameStrId,CashItem,Bionic,TypeId1,TypeId2,TypeId3,...,IsHwanPotion,IsHgpPotion,IsPet2SatietyPotion,IsRepairKit,IsArmor,IsShield,IsAccessory,IsWeapon,Degree,DegreeOffset
Key,,,,,,,,,,,,,,,,,,,,,
55,1,55,ITEM_ETC_CURE_ALL_01,?? ??(?),SN_ITEM_ETC_CURE_ALL_01,0,0,3,3,2,...,False,False,False,False,False,False,False,False,1.000000,0.0
56,1,56,ITEM_ETC_CURE_ALL_02,?? ??(?),SN_ITEM_ETC_CURE_ALL_02,0,0,3,3,2,...,False,False,False,False,False,False,False,False,1.333333,0.0
57,1,57,ITEM_ETC_CURE_ALL_03,?? ??(?),SN_ITEM_ETC_CURE_ALL_03,0,0,3,3,2,...,False,False,False,False,False,False,False,False,1.666667,0.0
58,1,58,ITEM_ETC_CURE_ALL_04,?? ?? ?? (?),SN_ITEM_ETC_CURE_ALL_04,0,0,3,3,2,...,False,False,False,False,False,False,False,False,2.000000,0.0
59,1,59,ITEM_ETC_CURE_ALL_05,?? ?? ?? (?),SN_ITEM_ETC_CURE_ALL_05,0,0,3,3,2,...,False,False,False,False,False,False,False,False,2.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,1,945,ITEM_CH_M_CLOTHES_04_LA_B,?? ???? ??,SN_ITEM_CH_CLOTHES_04_LA_B,0,0,3,1,1,...,False,False,False,False,False,False,False,False,4.333333,0.0
946,1,946,ITEM_CH_M_CLOTHES_04_LA_C,?? ???? ??,SN_ITEM_CH_CLOTHES_04_LA_C,0,0,3,1,1,...,False,False,False,False,False,False,False,False,4.666667,0.0
947,1,947,ITEM_CH_M_CLOTHES_05_LA_A,?? ??? ??,SN_ITEM_CH_CLOTHES_05_LA_A,0,0,3,1,1,...,False,False,False,False,False,False,False,False,5.000000,0.0


In [171]:
lines = get_lines(quest_reward_items)
for idx, line in enumerate(lines):
  values = line.split('\t')
  if len(values) < 11:
    continue
  
  QuestId = values[0]
  QuestCodeName = values[1]
  RewardType = values[2]
  ItemCodeName = values[3]
  OptionalItemCode = values[4]
  OptionalItemCount = values[5]
  AchieveQuantity = values[6]
  RentItemCodeName = values[7]
  # Item = None if ItemCodeName == 'xxx' else lookup_item(ItemCodeName)
  # OptionalItem = None if OptionalItemCode == 'xxx' else lookup_item(OptionalItemCode)
  # RentItem = None if RentItemCodeName == 'xxx' else lookup_item(RentItemCodeName)
  

  obj = {
    "QuestId": QuestId,
    "QuestCodeName": QuestCodeName,
    "RewardType": RewardType,
    "ItemCodeName": ItemCodeName,
    "OptionalItemCode": OptionalItemCode,
    "OptionalItemCount": OptionalItemCount,
    "AchieveQuantity": AchieveQuantity,
    "RentItemCodeName": RentItemCodeName
  }

  quest_reward_items_data.append(obj)

In [172]:
quest_reward_items_data

[{'QuestId': '123',
  'QuestCodeName': 'QNO_RM_VILLAGECHIEF_2',
  'RewardType': '0',
  'ItemCodeName': 'ITEM_QNO_RM_VILLAGECHIEF_2_01',
  'OptionalItemCode': 'xxx',
  'OptionalItemCount': 'xxx',
  'AchieveQuantity': '0',
  'RentItemCodeName': '1'},
 {'QuestId': '269',
  'QuestCodeName': 'QNO_TQ_MAGICPOWDER_3',
  'RewardType': '0',
  'ItemCodeName': 'ITEM_QNO_TQ_MAGICPOWDER_3_02',
  'OptionalItemCode': 'xxx',
  'OptionalItemCount': 'xxx',
  'AchieveQuantity': '0',
  'RentItemCodeName': '10'},
 {'QuestId': '277',
  'QuestCodeName': 'QNO_TQ_STONEBOARD_5',
  'RewardType': '0',
  'ItemCodeName': 'ITEM_QNO_TQ_STONEBOARD_5_02',
  'OptionalItemCode': 'xxx',
  'OptionalItemCount': 'xxx',
  'AchieveQuantity': '0',
  'RentItemCodeName': '1'},
 {'QuestId': '128',
  'QuestCodeName': 'QNO_RM_FLYSHIP2_2',
  'RewardType': '0',
  'ItemCodeName': 'ITEM_QNO_RM_FLYSHIP2_2_03',
  'OptionalItemCode': 'xxx',
  'OptionalItemCount': 'xxx',
  'AchieveQuantity': '0',
  'RentItemCodeName': '1'},
 {'QuestId': '128

## refqusetreward.txt

In [173]:
refqusetreward_file = os.path.join(server_dep, 'refqusetreward.txt')
refqusetreward_file

'server_dep\\silkroad\\textdata\\refqusetreward.txt'

In [174]:
lines = get_lines(refqusetreward_file)
for idx, line in enumerate(lines):
    try:
      values = line.split('\t')
      if len(values) < 11:
        continue

      QuestId = values[0],
      QuestCodeName = values[1],
      IsView = values[2],
      IsBasicReward = values[3],
      IsItemReward = values[4],
      IsCheckCondition = values[5],
      IsCheckCountry = values[6],
      IsCheckClass = values[7],
      IsCheckGender = values[8],

      Gold = values[10]
      Exp = values[11]
      SPExp = values[12]
      SP = values[13]
      AP = values[14]
      APType = values[15]
      Hwan = values[16]
      InventorySlots = values[17]
      ItemRewardType = values[18]
      SelectionCount = values[19]

      quest_rewards_data[QuestId] = {
        "QuestId": QuestId,
        "QuestCodeName": QuestCodeName,
        "IsView": IsView,
        "IsBasicReward": IsBasicReward,
        "IsItemReward": IsItemReward,
        "IsCheckCondition": IsCheckCondition,
        "IsCheckCountry": IsCheckCountry,
        "IsCheckClass": IsCheckClass,
        "IsCheckGender": IsCheckGender,
        "Gold": Gold,
        "Exp": Exp,
        "SPExp": SPExp,
        "SP": SP,
        "AP": AP,
        "APType": APType,
        "Hwan": Hwan,
        "InventorySlots": InventorySlots,
        "ItemRewardType": ItemRewardType,
        "SelectionCount": SelectionCount
      }
    except Exception as e:
      continue

In [175]:
quest_rewards_data

{('29',): {'QuestId': ('29',),
  'QuestCodeName': ('QSP_ALL_POTION_1',),
  'IsView': ('0',),
  'IsBasicReward': ('0',),
  'IsItemReward': ('0',),
  'IsCheckCondition': ('0',),
  'IsCheckCountry': ('0',),
  'IsCheckClass': ('0',),
  'IsCheckGender': ('0',),
  'Gold': '0',
  'Exp': '0',
  'SPExp': '0',
  'SP': '0',
  'AP': '0',
  'APType': 'xxx',
  'Hwan': '0',
  'InventorySlots': '0',
  'ItemRewardType': '0',
  'SelectionCount': '0'},
 ('38',): {'QuestId': ('38',),
  'QuestCodeName': ('QNO_KT_SOLDIER_EA2_1',),
  'IsView': ('1',),
  'IsBasicReward': ('1',),
  'IsItemReward': ('0',),
  'IsCheckCondition': ('0',),
  'IsCheckCountry': ('0',),
  'IsCheckClass': ('0',),
  'IsCheckGender': ('0',),
  'Gold': '0',
  'Exp': '5975100',
  'SPExp': '0',
  'SP': '0',
  'AP': '0',
  'APType': 'xxx',
  'Hwan': '0',
  'InventorySlots': '0',
  'ItemRewardType': '0',
  'SelectionCount': '0'},
 ('39',): {'QuestId': ('39',),
  'QuestCodeName': ('QNO_KT_ACCESSORY_1',),
  'IsView': ('1',),
  'IsBasicReward': 

## QuestData.txt

In [176]:
quest_data_file = os.path.join(server_dep, 'QuestData.txt')
quest_data_file

'server_dep\\silkroad\\textdata\\QuestData.txt'

In [177]:
quest_data_files = get_lines(quest_data_file)
quest_data_files

['QuestData_1000.txt',
 'QuestData_1250.txt',
 'QuestData_1500.txt',
 'QuestData_1750.txt',
 'QuestData_2000.txt',
 'QuestData_250.txt',
 'QuestData_500.txt',
 'QuestData_750.txt',
 '']

In [178]:
for xxx in quest_data_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    print(line)
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = values[0]
    ID = values[1]
    CodeName = values[2]
    Level = values[3]
    DescName = values[4]
    NameString = values[5]
    PayString = values[6]
    ContentsString = values[7]
    PayContents = values[8]
    NoticeNPC = values[9]
    NoticeCondition = values[10]
    # Reward = quest_rewards_data[ID]
    # RewardItems = [x for x in quest_reward_items_data if x['QuestId'] == ID]

    quest_data[ID] = {
      "Service": Service,
      "ID": ID,
      "CodeName": CodeName,
      "Level": Level,
      "DescName": DescName,
      "NameString": NameString,
      "PayString": PayString,
      "ContentsString": ContentsString,
      "PayContents": PayContents,
      "NoticeNPC": NoticeNPC,
      "NoticeCondition": NoticeCondition
    }


1	750	QNO_FW_RB2_012	36	<??? ??> ???? ?? ??	SN_QNO_FW_RB2_012	SN_PAY_QNO_FW_RB2_012	xxx	SN_PAYCON_QNO_FW_RB2_012	SN_NN_QNO_FW_RB2_012	SN_NC_QNO_FW_RB2_012
1	751	QNO_FW_RB2_013	36	<??? ??> ?? ??? ??	SN_QNO_FW_RB2_013	SN_PAY_QNO_FW_RB2_013	xxx	SN_PAYCON_QNO_FW_RB2_013	SN_NN_QNO_FW_RB2_013	SN_NC_QNO_FW_RB2_013
1	752	QNO_FW_RB2_014	39	<??? ??> ??? ??	SN_QNO_FW_RB2_014	SN_PAY_QNO_FW_RB2_014	xxx	SN_PAYCON_QNO_FW_RB2_014	SN_NN_QNO_FW_RB2_014	SN_NC_QNO_FW_RB2_014
1	753	QNO_FW_RB2_015	51	<??? ??> ?? ?????…	SN_QNO_FW_RB2_015	SN_PAY_QNO_FW_RB2_015	xxx	SN_PAYCON_QNO_FW_RB2_015	SN_NN_QNO_FW_RB2_015	SN_NC_QNO_FW_RB2_015
1	754	QNO_FW_RB2_016	52	<??? ??> ??? ??	SN_QNO_FW_RB2_016	SN_PAY_QNO_FW_RB2_016	xxx	SN_PAYCON_QNO_FW_RB2_016	SN_NN_QNO_FW_RB2_016	SN_NC_QNO_FW_RB2_016
1	755	QNO_FW_RB2_017	51	<??? ??> ?? ??	SN_QNO_FW_RB2_017	SN_PAY_QNO_FW_RB2_017	xxx	SN_PAYCON_QNO_FW_RB2_017	SN_NN_QNO_FW_RB2_017	SN_NC_QNO_FW_RB2_017
1	756	QNO_FW_RB2_018	61	<??? ??> ??? ???? ??	SN_QNO_FW_RB2_018	SN_PAY_QNO_FW_RB2_018	

In [179]:
quest_data

{'750': {'Service': '1',
  'ID': '750',
  'CodeName': 'QNO_FW_RB2_012',
  'Level': '36',
  'DescName': '<??? ??> ???? ?? ??',
  'NameString': 'SN_QNO_FW_RB2_012',
  'PayString': 'SN_PAY_QNO_FW_RB2_012',
  'ContentsString': 'xxx',
  'PayContents': 'SN_PAYCON_QNO_FW_RB2_012',
  'NoticeNPC': 'SN_NN_QNO_FW_RB2_012',
  'NoticeCondition': 'SN_NC_QNO_FW_RB2_012'},
 '751': {'Service': '1',
  'ID': '751',
  'CodeName': 'QNO_FW_RB2_013',
  'Level': '36',
  'DescName': '<??? ??> ?? ??? ??',
  'NameString': 'SN_QNO_FW_RB2_013',
  'PayString': 'SN_PAY_QNO_FW_RB2_013',
  'ContentsString': 'xxx',
  'PayContents': 'SN_PAYCON_QNO_FW_RB2_013',
  'NoticeNPC': 'SN_NN_QNO_FW_RB2_013',
  'NoticeCondition': 'SN_NC_QNO_FW_RB2_013'},
 '752': {'Service': '1',
  'ID': '752',
  'CodeName': 'QNO_FW_RB2_014',
  'Level': '39',
  'DescName': '<??? ??> ??? ??',
  'NameString': 'SN_QNO_FW_RB2_014',
  'PayString': 'SN_PAY_QNO_FW_RB2_014',
  'ContentsString': 'xxx',
  'PayContents': 'SN_PAYCON_QNO_FW_RB2_014',
  'NoticeN